Business count by Age of Business

In [1]:
from databaker.framework import *
import pandas as pd

In [2]:
import requests
from pathlib import Path

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.uktradeinfo.com/statistics/documents/idbr_ots_tables_2015.xls'
inputFile = sourceFolder / 'idbr_ots_tables_2015.xls'
if not(inputFile.exists() and inputFile.is_file()):
    response = requests.get(inputURL)
    with open(inputFile, 'wb') as f:
        f.write(response.content)

In [3]:
tab = loadxlstabs(inputFile)

Loading in\idbr_ots_tables_2015.xls which has size 174080 bytes
Table names: ['Notes and Contents', 'Industry Group', 'Age Group', 'Employee Size', 'Industry_Age', 'Industry_EmployeeSize', 'EmployeeSize_Age', 'Metadata']


In [4]:
tab = tab[2]

In [5]:
savepreviewhtml(tab)

In [6]:
observations = tab.excel_ref('C25').expand(DOWN).expand(RIGHT).is_not_blank()-tab.excel_ref('C24').expand(UP)

In [7]:
observations = observations - tab.excel_ref('C34').expand(DOWN).expand(RIGHT)

In [8]:
Businessage = tab.excel_ref('A').expand(DOWN)

In [13]:
Flow = tab.excel_ref('C5').expand(RIGHT)

In [19]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDimConst('Year','2015'),
            HDimConst('Unit', 'Count'), 
            HDimConst('Measure Type','Business'),            
            HDim(Businessage, 'Business Age(Yrs)', DIRECTLY, LEFT),
            HDim(Flow, 'Flow', CLOSEST, LEFT)    
]

In [20]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [21]:
savepreviewhtml(c1)

In [22]:
new_table = c1.topandas()

In [23]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)

In [29]:
new_table['Value'] = new_table['Value'].astype('int', copy=False)

In [30]:
new_table['Business Age(Yrs)'] = new_table['Business Age(Yrs)'].str.strip()
new_table['Year'] = new_table['Year'].str.strip()
new_table['Flow'] = new_table['Flow'].str.strip()
new_table['Unit'] = new_table['Unit'].str.strip()
new_table['Measure Type'] = new_table['Measure Type'].str.strip()
new_table['Geography'] = new_table['Geography'].str.strip()

In [31]:
new_table = new_table[['Geography','Year','Business Age(Yrs)','Flow','Measure Type','Value','Unit']]

In [32]:
new_table.head(4)

Geography  Year Business Age(Yrs) Flow Measure Type  Value   Unit
0  K02000001  2015            0 to 1          Business  10515  Count
1  K02000001  2015            0 to 1          Business  21541  Count
2  K02000001  2015            2 to 3          Business  13308  Count
3  K02000001  2015            2 to 3          Business  22043  Count

In [33]:
new_table.tail(4)

Geography  Year Business Age(Yrs) Flow Measure Type   Value   Unit
12  K02000001  2015           Unknown          Business   10246  Count
13  K02000001  2015           Unknown          Business   25316  Count
14  K02000001  2015             Total          Business  143358  Count
15  K02000001  2015             Total          Business  219302  Count

In [34]:
out = Path('out')
out.mkdir(exist_ok=True)
new_table.to_csv(out / 'BusinesscountbyAgeofBusiness.csv', index = False)